# School meals

School meal subsidies comprise the National School Lunch Program (NSLP) and the School Breakfast Program (SBP).
Eligibility depends on household size, state, income, and participation in other programs, and assigns children to either a *reduced* or *free* school meal subsidy tier.

## Eligibility

Children whose families have income below 130 percent of the federal poverty line, or who participate in SNAP, TANF, or the Food Distribution Program on Indian Reservations, are eligible for free breakfast and lunch at school.
Children who do not meet these criteria and whose families have income below 185 percent of the federal poverty line are eligible for a reduced school meal subsidy.
Other children receive a small meal subsidy from the federal government.

## Benefit value

We assign the benefit value as the federal reimbursement rates published by USDA.
We then multiply this by 180 school days per year.
We subtract the subsidy value provided to all children.

## Examples

### One child

Consider a one-adult one-child household in California with \$1,000 monthly earned income.
They would be eligible for \$74 per month in net school meal subsidies.

In [ ]:
from policyengine_us import Simulation
import pandas as pd
import plotly.express as px

# Create a one-adult, one-child household
situation = {
    "people": {
        "parent": {
            "employment_income": {"2022": 1000 * 12}
        },
        "child": {
            "age": {"2022": 10},
            "is_in_k12_school": {"2022": True}
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["parent", "child"],
            "snap": {"2022": 0}
        }
    },
    "households": {
        "household": {
            "members": ["parent", "child"],
            "state_name": {"2022": "CA"}
        }
    }
}

sim = Simulation(situation=situation)
school_meal_subsidy = sim.calculate("school_meal_net_subsidy", 2022)[0]
round(school_meal_subsidy / 12)

What if their earnings change?
They receive the maximum benefit of \$74 per month until they reach \$1,880 in monthly earnings, at which point it falls to \$64 per month until they reach \$2,690 per month, at which point it falls to \$0.

In [ ]:
# Calculate for different income levels
results = []
for monthly_income in range(0, 4000, 10):
    annual_income = monthly_income * 12
    
    situation = {
        "people": {
            "parent": {
                "employment_income": {"2022": annual_income}
            },
            "child": {
                "age": {"2022": 10},
                "is_in_k12_school": {"2022": True}
            }
        },
        "spm_units": {
            "spm_unit": {
                "members": ["parent", "child"],
                "snap": {"2022": 0}
            }
        },
        "households": {
            "household": {
                "members": ["parent", "child"],
                "state_name": {"2022": "CA"}
            }
        }
    }
    
    sim = Simulation(situation=situation)
    subsidy = sim.calculate("school_meal_net_subsidy", 2022)[0]
    
    results.append({
        "employment_income": monthly_income,
        "school_meal_net_subsidy": round(subsidy / 12)
    })

df = pd.DataFrame(results)

LABELS = dict(
    employment_income="Monthly employment income",
    school_meal_net_subsidy="Monthly school meal subsidy",
    mtr="Marginal tax rate",
)

fig = px.line(
    df,
    "employment_income",
    "school_meal_net_subsidy",
    labels=LABELS,
    title="School meal subsidy for a one-parent one-child household in California",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()